## Assignment 4
#### Group 1: Tara Bode, Hankun Li

Part A: Text-based industry classification
Cluster the business sections extracted in assignment 3 into 50 clusters (using the cosine similarity) to group firms with similar business descriptions together.

Submit your code and the final output csv file, with CIK, conformed end of period, and the cluster number for each business section file.

Note: it is likely you will need to tokenize/word count each business section file before reading the next one. (If you try to read all business sections before doing any further processing you will probably run out of memory).

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

You need to simply take the.txt files from HiperGator and use the path (r'/blue/acg7849/share/BS/') 

so we can grab the length of each file for the csv file.

Also, we need to use the for loop to cluster the business section files and mimic the class code from yesterday to grab the top 20 common words.

# all functions

In [2]:
def getHeaderVariables(filing):
    CIK = filing[ filing.find('CENTRAL INDEX KEY:') + 18  : filing.find('STANDARD') ]
    CIK = ''.join(CIK.split())
    #
    CCN = filing[ filing.find('COMPANY CONFORMED NAME:') + 23 : filing.find('CENTRAL INDEX KEY') ]
    CCN = CCN.rstrip().lstrip()
    #
    CST = filing[ filing.find('CONFORMED SUBMISSION TYPE:') + 26 : filing.find('PUBLIC DOCUMENT COUNT') ]
    CST = ''.join(CST.split())
    #
    CPR = filing[ filing.find('CONFORMED PERIOD OF REPORT:') + 27 : filing.find('FILED AS OF DATE') ]
    CPR = ''.join(CPR.split())
    #
    Length = 'skip'
    
    sec_info = '{} | {} | {} | {} | {}\n'.format(CIK, CCN, CST, CPR, Length)
    return (sec_info)


In [3]:
def cleanHTML(raw_html):
    cleanr = re.compile('<.*?>', flags=re.DOTALL)
    cleantext = re.sub(cleanr, '', raw_html)
    return replace_entities(cleantext)

In [4]:
def getBusinessSection(filing):
    # USE REGEXR
    BDS = filing[ filing.find('Item 1. ') : filing.find('Item 1A. ') ]
    #print('length_business_section',len(BDS))
    #print(BDS)
    return BDS
    #

# loop in A3 - successfully inherited - not preferred

csv file, with CIK, conformed end of period, and file name (business section file) and the cluster number for each business section file.

In [5]:
mydirectory = '/blue/acg7849/share/10Ks'
file_list = glob.glob(mydirectory + '/*.txt')
#print(file_list)

In [6]:
# write header variables and length of business section to disk
with open (r'/blue/acg7849/hli1/SEC_Information.csv', 'w') as f:
    f.write('File Name | Central Index Key | Company Name | Submission Type | Period of Report | Length of Business Description\n')

for file_path in file_list[0:10]: # use 10 files to test
    # read the file
    with open(file_path) as f:
        filing = f.read()
        
    # only use until /DOCUMENT
    first_section = filing[0: filing.find('</DOCUMENT')]
    
    # get header variables
    headervars = str(getHeaderVariables(first_section))
    
    # isolate business section
    business_section = str(getBusinessSection(first_section))
    
    #need to input FileName
    shortName = file_path[file_path.rfind("/")+1:]
    
    # write business section to disk       
    with open(r'/blue/acg7849/hli1/item1/'+ shortName, 'w', encoding="utf-8") as f:
        f.write(cleanHTML(business_section))
   
    with open (r'/blue/acg7849/hli1/SEC_Information.csv', 'a', encoding="utf-8") as f:
        f.write(shortName + ' | ' + cleanHTML(headervars))

    #with open (r'/blue/acg7849/tbode/SEC_Information.csv', 'a', encoding="utf-8") as f:
     #   f.write(shortName + ' | ' + cleanHTML(headervars) + ' | ' + len(cleanHTML(business_section)))


 # recommended - use summary files

In [70]:
# matching filenames

with open ('/blue/acg7849/share/BS/summary.txt') as a: 
    summary = a. readlines()
print (summary[0:5])

['CIK|coname|formtype|date|filename|length\n', '0001289850|NeuroMetrix, Inc.|10-K|20171231|267762.txt|132616\n', '0001345016|YELP INC|10-K|20191231|280603.txt|152323\n', '0001245791|GI DYNAMICS, INC.|10-K|20171231|267435.txt|28236\n', '0001772177|KURA SUSHI USA, INC.|10-K|20190831|275323.txt|38468\n']


# new loop for clustering

In [7]:
newdirectory = '/blue/acg7849/share/BS/item1'
file_list1 = glob.glob(newdirectory + '/*.txt')
#print(file_list1) # success!

In [8]:
obj = pd.DataFrame(file_list)
obj.to_csv('file_list.csv', index = True) # csv preapared to use

In [9]:
badFiles_raw = []

In [10]:
for i in file_list[0:5]:
    print (i)
    # read the file
    with open(i) as f:
        filing = f.read()
        
    # grab the length of the file
    print (len(filing))

    # Ignore business section files with less than 1000 characters.
    if len(filing) < 50000: # test. should be 1000 here
        print ('bad')
        badFiles_raw.append (i)
    # should end this file and return to next one
    # No. of cluster should be 0 or NA

    
    #need to input FileName
    shortName = i[i.rfind("/")+1:]
    print(shortName)
    # match shortnames to cluster No.
    
    # tokenize
    #word_tokenize (filing[0:10]) # bit different from split - include punctuations
#print(badFiles_raw)

/blue/acg7849/share/10Ks/267762.txt
7858323
267762.txt
/blue/acg7849/share/10Ks/280603.txt
15640951
280603.txt
/blue/acg7849/share/10Ks/267435.txt
9945879
267435.txt
/blue/acg7849/share/10Ks/275323.txt
3221064
275323.txt
/blue/acg7849/share/10Ks/277539.txt
15953715
277539.txt


In [11]:
print(badFiles_raw)

[]


In [12]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [13]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [14]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [15]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

## make a list of full texts to do K-means
## reference: http://brandonrose.org/clustering

In [90]:
totalvocab_stemmed = []
totalvocab_tokenized = []

newdirectory = '/blue/acg7849/share/BS/item1'
file_list1 = glob.glob(newdirectory + '/*.txt')
#print(file_list1) # success!

filing1 = []
n=-1

for i in file_list1[0:50]:
    print (i)
    # read the file
    with open(i) as f:
        #filing = f.read()
        filing = f.readlines() # some were one line and some were multi lines...so need combine
        print("num of elem:"len(filing))
        print(n)
        n = n +1
        if len(filing) > 1:
            print(n) # this is the (n-1)th file
            filing[n : 99999] = [''.join(filing[n : 99999])]
            #print("this is",filing[-1]) # seems that the last element is involved
        filing1 += filing
print(len(filing1))  # there can be a big number so the last one is not looping

/blue/acg7849/share/BS/item1/267762.txt
1
-1
/blue/acg7849/share/BS/item1/280603.txt
1
0
/blue/acg7849/share/BS/item1/267435.txt
489
1
2
/blue/acg7849/share/BS/item1/275323.txt
176
2
3
/blue/acg7849/share/BS/item1/277539.txt
1
3
/blue/acg7849/share/BS/item1/270762.txt
2035
4
5
/blue/acg7849/share/BS/item1/274077.txt
9
5
6
/blue/acg7849/share/BS/item1/265478.txt
1
6
/blue/acg7849/share/BS/item1/280595.txt
1685
7
8
/blue/acg7849/share/BS/item1/283125.txt
11
8
9
/blue/acg7849/share/BS/item1/281916.txt
4
9
10
/blue/acg7849/share/BS/item1/282859.txt
2121
10
11
/blue/acg7849/share/BS/item1/284462.txt
1
11
/blue/acg7849/share/BS/item1/270024.txt
1
12
/blue/acg7849/share/BS/item1/280098.txt
747
13
14
/blue/acg7849/share/BS/item1/280672.txt
3
14
15
/blue/acg7849/share/BS/item1/274227.txt
2
15
16
/blue/acg7849/share/BS/item1/266806.txt
1
16
/blue/acg7849/share/BS/item1/283268.txt
1
17
/blue/acg7849/share/BS/item1/266373.txt
649
18
19
/blue/acg7849/share/BS/item1/269026.txt
1042
19
20
/blue/acg78

In [17]:
type(filing1)

list

In [75]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in filing1:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'filing1', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [76]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 72841 items in vocab_frame


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(filing1) #fit the vectorizer to a iterable list of txt

print(tfidf_matrix.shape)

/home/hli1/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CPU times: user 1.77 s, sys: 4.22 ms, total: 1.78 s
Wall time: 1.79 s
(49, 170)


In [78]:
terms = tfidf_vectorizer.get_feature_names()

In [79]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [80]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# KMC

In [81]:
from sklearn.cluster import KMeans

num_clusters = 5                            # doc num > num of clst

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 62 ms, sys: 0 ns, total: 62 ms
Wall time: 63.6 ms


In [82]:
import joblib
#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle
joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [83]:
Comps = { 'shortName': shortName, 'cluster': clusters}
frame = pd.DataFrame(Comps, index = [clusters] , columns = ['shortName', 'cluster'])

In [84]:
print(frame) # why do all the short names same? seem improper

    shortName  cluster
1  277539.txt        1
1  277539.txt        1
2  277539.txt        2
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
3  277539.txt        3
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
1  277539.txt        1
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
2  277539.txt        2
3  277539.txt        3
0  277539.txt        0
0  277539.txt        0
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
4  277539.txt        4
4  277539.txt        4
4  277539.txt        4
4  277539.txt        4
4  277539.t

In [85]:
frame['cluster'].value_counts()

0    24
4    10
1     8
2     5
3     2
Name: cluster, dtype: int64

In [86]:
# which cluster are the comps in accordingly?